In [1]:
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import random
import warnings
from datetime import timedelta

1. Прочитайте базу my_BRENT2019.csv и перейдите к представлению ряда первыми разностями
2. Добавьте первые разности по стоимости доллара и евро к рублю
3. сделайте ее перрасчет (ресемплинг) в представление по неделям. 
4. Постройте модель предсказания 1-й точки от текущей (h=1).

5. Сделайте вывод о поведении ошибки предсказания ряда в сравнении с прямым прогнозированием.

1. Прочитайте базу my_BRENT2019.csv и перейдите к представлению ряда первыми разностями

In [2]:
path_brent = 'my_BRENT2019.csv'
brent_data = pd.read_csv(path_brent,sep = ',', index_col=[0], parse_dates=[0])
brent_data.head()

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday
Дата,,,,,,,,,,,
2010-09-01,76.37,2010,9,1,3,244,35,2,0,0,0
2010-09-02,76.89,2010,9,2,3,245,35,3,0,0,0
2010-09-03,76.57,2010,9,3,3,246,35,4,0,0,0
2010-09-04,76.62,2010,9,4,3,247,35,5,0,0,0
2010-09-05,76.63,2010,9,5,3,248,35,6,0,1,0


In [3]:
full_list_date = pd.date_range(brent_data.index.min(), brent_data.index.max(), freq="D")
diff_list = set(full_list_date) - set(brent_data.index.unique())
print(f'Количество пропущенных дней: {len(diff_list)}')

Количество пропущенных дней: 621


In [4]:
full_data = pd.DataFrame(full_list_date, columns=['Date'], index=full_list_date)
full_data = full_data.join(brent_data, on='Date', how='left')
full_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3288 entries, 2010-09-01 to 2019-09-01
Freq: D
Data columns (total 12 columns):
Date               3288 non-null datetime64[ns]
Значение           2667 non-null float64
year               2667 non-null float64
month              2667 non-null float64
dayofmonth         2667 non-null float64
quarter            2667 non-null float64
dayofyear          2667 non-null float64
weekofyear         2667 non-null float64
dayofweek          2667 non-null float64
is_holiday         2667 non-null float64
is_prev_holiday    2667 non-null float64
is_next_holiday    2667 non-null float64
dtypes: datetime64[ns](1), float64(11)
memory usage: 333.9 KB


In [5]:
from datetime import timedelta
one_day = timedelta(days=1)

mas = full_data.loc[full_data['Значение'].isna(), ['Значение']].index



for i in mas:    
    min_day_value = None
    min_day = i
    while(True):
        min_day = min_day - one_day
        min_day_value = full_data.loc[min_day, ['Значение']].values[0]
        if min_day_value > 1:
            break;
            
    max_day_value = None
    max_day = i
    while(True):
        max_day = max_day + one_day
        max_day_value = full_data.loc[max_day, ['Значение']].values[0]
        if max_day_value > 1:
            break;
            
    full_data.loc[i, ['Значение']] = (max_day_value + min_day_value) / 2
    full_data.loc[i, ['year']] = i.year
    full_data.loc[i, ['month']] = i.month
    full_data.loc[i, ['dayofmonth']] = i.day
    full_data.loc[i, ['quarter']] = i.quarter
    full_data.loc[i, ['dayofyear']] = i.dayofyear
    full_data.loc[i, ['weekofyear']] = i.weekofyear
    full_data.loc[i, ['dayofweek']] = i.dayofweek
    full_data.loc[i, ['is_holiday']] = 0
    full_data.loc[i, ['is_prev_holiday']] = 0
    full_data.loc[i, ['is_next_holiday']] = 0

In [6]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3288 entries, 2010-09-01 to 2019-09-01
Freq: D
Data columns (total 12 columns):
Date               3288 non-null datetime64[ns]
Значение           3288 non-null float64
year               3288 non-null float64
month              3288 non-null float64
dayofmonth         3288 non-null float64
quarter            3288 non-null float64
dayofyear          3288 non-null float64
weekofyear         3288 non-null float64
dayofweek          3288 non-null float64
is_holiday         3288 non-null float64
is_prev_holiday    3288 non-null float64
is_next_holiday    3288 non-null float64
dtypes: datetime64[ns](1), float64(11)
memory usage: 461.9 KB


In [7]:
diff1 = np.array(full_data.iloc[1:,1].values - full_data.iloc[:-1,1].values).astype('float32') 
full_data['diff1'] = 0.0
full_data['diff1'] = full_data['diff1'].astype('float') 
full_data['diff1'].values[1:] = diff1

In [8]:
full_data.head()

,Date,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday,diff1
2010-09-01,2010-09-01,76.37,2010.0,9.0,1.0,3.0,244.0,35.0,2.0,0.0,0.0,0.0,0.00
2010-09-02,2010-09-02,76.89,2010.0,9.0,2.0,3.0,245.0,35.0,3.0,0.0,0.0,0.0,0.52
2010-09-03,2010-09-03,76.57,2010.0,9.0,3.0,3.0,246.0,35.0,4.0,0.0,0.0,0.0,-0.32
2010-09-04,2010-09-04,76.62,2010.0,9.0,4.0,3.0,247.0,35.0,5.0,0.0,0.0,0.0,0.05
2010-09-05,2010-09-05,76.63,2010.0,9.0,5.0,3.0,248.0,35.0,6.0,0.0,1.0,0.0,0.01


2. Добавьте первые разности по стоимости доллара и евро к рублю

In [109]:
USD = pd.read_csv('RC_F01_09_2010_T01_09_2019_USD.csv', sep=';', index_col=[0], parse_dates=[0])
USD['curs'] = USD['curs'].str.replace(',','.',regex=True)
USD['curs'] = USD['curs'].astype('float64')

diff_list_USD = set(full_list_date) - set(USD.index.unique())
print(f'Количество пропущенных дней: {len(diff_list_USD)}')

USD = pd.concat([pd.DataFrame(full_list_date, index=full_list_date), USD], axis=1)

USD = USD.loc[USD.index <= '2019-09-01']

mas = USD.loc[USD['curs'].isna(), ['curs']].index
for i in mas:    
    min_day_value = None
    min_day = i
    while(True):
        min_day = min_day - one_day
        min_day_value = USD.loc[min_day, ['curs']].values[0]
        if min_day_value > 1:
            break;
            
    max_day_value = None
    max_day = i
    while(True):
        max_day = max_day + one_day
        max_day_value = USD.loc[max_day, ['curs']].values[0]
        if max_day_value > 1:
            break;
            
USD.info()

Количество пропущенных дней: 1100


KeyError: Timestamp('2010-08-31 00:00:00')

In [26]:
EUR_diff1 = np.array(EUR.iloc[1:,0].values - EUR.iloc[:-1,0].values).astype('float32') 
USD_diff1 = np.array(USD.iloc[1:,0].values - USD.iloc[:-1,0].values).astype('float32') 

TypeError: unsupported operand type(s) for -: 'str' and 'str'